In [79]:
import pandas as pd
raw_dataset=pd.read_csv("../Datasets/Clustering/Bank Customer Segmentation/raw_dataset.csv")

In [80]:
raw_dataset.drop(['TransactionID','CustomerID'], axis = 1, inplace=True)

In [81]:
raw_dataset

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207.0,25.0
1,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858.0,27999.0
2,26/11/96,F,MUMBAI,17874.44,2/8/16,142712.0,459.0
3,14/9/73,F,MUMBAI,866503.21,2/8/16,142714.0,2060.0
4,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156.0,1762.5
...,...,...,...,...,...,...,...
748957,14/3/75,M,MUMBAI,84485.71,1/9/16,214431.0,10000.0
748958,1/1/1800,M,DELHI,8425.13,1/9/16,211447.0,300.0
748959,17/8/91,M,JAMNAGAR,57887.14,1/9/16,214610.0,159.0
748960,1/1/1800,M,NEW DELHI,66238.54,1/9/16,215233.0,1319.0


### 1.Handling TransactionTime & TransactionAmount (INR) Column. 

Transaction amount null value will not help in clustering the customers. Hence dropping the row with null value in TransactionAmount (INR) column

In [82]:
raw_dataset = raw_dataset.dropna(subset=['TransactionAmount (INR)'])
raw_dataset.shape

(748961, 7)

In [83]:
raw_dataset.isna().sum()

CustomerDOB                2451
CustGender                  814
CustLocation                116
CustAccountBalance         1885
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [84]:
import pandas as pd
from datetime import datetime

# Example: Assume you already have a DataFrame
# raw_dataset = pd.read_csv("your_file.csv")  # If you're reading from a file

# Convert the 'TransactionTime' column (Unix timestamps) to HH:MM:SS
raw_dataset['TransactionTime'] = pd.to_datetime(raw_dataset['TransactionTime'], unit='s')

# Format it as HH:MM:SS only
raw_dataset['TransactionTime'] = raw_dataset['TransactionTime'].dt.strftime('%H:%M:%S')

# View updated data
print(raw_dataset[['TransactionTime']].head())


  TransactionTime
0        15:46:47
1        15:24:18
2        15:38:32
3        15:38:34
4        02:19:16


In [85]:
# Extract hour and minute as new columns
raw_dataset['Hour'] = pd.to_datetime(raw_dataset['TransactionTime'], format='%H:%M:%S').dt.hour
raw_dataset['Minute'] = pd.to_datetime(raw_dataset['TransactionTime'], format='%H:%M:%S').dt.minute

In [86]:
raw_dataset.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Hour,Minute
0,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,15:46:47,25.0,15,46
1,4/4/57,M,JHAJJAR,2270.69,2/8/16,15:24:18,27999.0,15,24
2,26/11/96,F,MUMBAI,17874.44,2/8/16,15:38:32,459.0,15,38
3,14/9/73,F,MUMBAI,866503.21,2/8/16,15:38:34,2060.0,15,38
4,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,02:19:16,1762.5,2,19


In [87]:
def map_time_period(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

raw_dataset['TimePeriod'] = raw_dataset['Hour'].apply(map_time_period)


In [88]:
raw_dataset.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Hour,Minute,TimePeriod
0,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,15:46:47,25.0,15,46,Afternoon
1,4/4/57,M,JHAJJAR,2270.69,2/8/16,15:24:18,27999.0,15,24,Afternoon
2,26/11/96,F,MUMBAI,17874.44,2/8/16,15:38:32,459.0,15,38,Afternoon
3,14/9/73,F,MUMBAI,866503.21,2/8/16,15:38:34,2060.0,15,38,Afternoon
4,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,02:19:16,1762.5,2,19,Night


In [89]:
raw_dataset['HourMinute'] = raw_dataset['TransactionTime'].str.slice(0, 5)  # e.g., "15:46"


In [90]:
raw_dataset.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Hour,Minute,TimePeriod,HourMinute
0,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,15:46:47,25.0,15,46,Afternoon,15:46
1,4/4/57,M,JHAJJAR,2270.69,2/8/16,15:24:18,27999.0,15,24,Afternoon,15:24
2,26/11/96,F,MUMBAI,17874.44,2/8/16,15:38:32,459.0,15,38,Afternoon,15:38
3,14/9/73,F,MUMBAI,866503.21,2/8/16,15:38:34,2060.0,15,38,Afternoon,15:38
4,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,02:19:16,1762.5,2,19,Night,02:19


### 2. Handling CustAccountBalance Column

In [91]:
mean_balance = raw_dataset['CustAccountBalance'].mean()
raw_dataset['CustAccountBalance'].fillna(mean_balance,inplace=True)

In [92]:
raw_dataset.isna().sum()

CustomerDOB                2451
CustGender                  814
CustLocation                116
CustAccountBalance            0
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
Hour                          0
Minute                        0
TimePeriod                    0
HourMinute                    0
dtype: int64

### 3.Handling CustGender Column & CustLocation
Replacing the custgender and CustLocation column with the mode value

In [93]:
raw_dataset['CustGender'].fillna(raw_dataset['CustGender'].mode()[0], inplace=True)
raw_dataset['CustLocation'].fillna(raw_dataset['CustLocation'].mode()[0], inplace=True)

In [94]:
raw_dataset.isna().sum()

CustomerDOB                2451
CustGender                    0
CustLocation                  0
CustAccountBalance            0
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
Hour                          0
Minute                        0
TimePeriod                    0
HourMinute                    0
dtype: int64

### 4.Handling CustomerDOB column

In [95]:
import warnings

# Ignore any warnings
warnings.filterwarnings("ignore")

In [96]:
# Convert CustomerDOB to datetime
raw_dataset['CustomerDOB'] = pd.to_datetime(raw_dataset['CustomerDOB'], errors='coerce')

In [97]:
# Create the Age Column
today = pd.Timestamp.today()
raw_dataset['Age'] = (today - raw_dataset['CustomerDOB']).dt.days // 365

In [98]:
# Handle Missing Age Values
raw_dataset['Age'].isna().sum()

median_age = raw_dataset['Age'].median()
raw_dataset['Age'].fillna(median_age, inplace=True)

In [99]:
# Convert 'Age' column to integers
raw_dataset['Age'] = raw_dataset['Age'].astype(int)

# Now, 'Age' will be an integer type

In [100]:
raw_dataset

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Hour,Minute,TimePeriod,HourMinute,Age
0,1994-10-01,F,JAMSHEDPUR,17819.05,2/8/16,15:46:47,25.0,15,46,Afternoon,15:46,30
1,2057-04-04,M,JHAJJAR,2270.69,2/8/16,15:24:18,27999.0,15,24,Afternoon,15:24,-32
2,1996-11-26,F,MUMBAI,17874.44,2/8/16,15:38:32,459.0,15,38,Afternoon,15:38,28
3,2073-09-14,F,MUMBAI,866503.21,2/8/16,15:38:34,2060.0,15,38,Afternoon,15:38,-49
4,1988-03-24,F,NAVI MUMBAI,6714.43,2/8/16,02:19:16,1762.5,2,19,Night,02:19,37
...,...,...,...,...,...,...,...,...,...,...,...,...
748956,1995-01-01,M,GURGAON,372.17,1/9/16,11:02:18,220.0,11,2,Morning,11:02,30
748957,1975-03-14,M,MUMBAI,84485.71,1/9/16,11:33:51,10000.0,11,33,Morning,11:33,50
748958,1800-01-01,M,DELHI,8425.13,1/9/16,10:44:07,300.0,10,44,Morning,10:44,225
748959,1991-08-17,M,JAMNAGAR,57887.14,1/9/16,11:36:50,159.0,11,36,Morning,11:36,33


In [101]:
# Drop rows where 'Age' column has negative values in place
raw_dataset.drop(raw_dataset[raw_dataset['Age'] < 0].index, inplace=True)

In [102]:
# Drop rows where 'Age' column has  values more than 110
raw_dataset.drop(raw_dataset[raw_dataset['Age'] > 110].index, inplace=True)

In [105]:
raw_dataset.dtypes

CustomerDOB                datetime64[ns]
CustGender                         object
CustLocation                       object
CustAccountBalance                float64
TransactionDate                    object
TransactionTime                    object
TransactionAmount (INR)           float64
Hour                                int64
Minute                              int64
TimePeriod                         object
HourMinute                         object
Age                                 int32
dtype: object

### 5.Changing datatypes

In [106]:
# Convert TransactionDate to datetime
raw_dataset['TransactionDate'] = pd.to_datetime(raw_dataset['TransactionDate'], errors='coerce')

# Convert TransactionTime to datetime.time format
raw_dataset['TransactionTime'] = pd.to_datetime(raw_dataset['TransactionTime'], format='%H:%M:%S', errors='coerce').dt.time

# Convert TimePeriod to category
raw_dataset['TimePeriod'] = raw_dataset['TimePeriod'].astype('category')

# Convert HourMinute to datetime or timedelta (if it's in HH:MM format)
# If HourMinute is like '14:30', convert to timedelta for easier plotting
raw_dataset['HourMinute'] = pd.to_timedelta(raw_dataset['HourMinute'] + ':00')


In [107]:
raw_dataset.dtypes

CustomerDOB                 datetime64[ns]
CustGender                          object
CustLocation                        object
CustAccountBalance                 float64
TransactionDate             datetime64[ns]
TransactionTime                     object
TransactionAmount (INR)            float64
Hour                                 int64
Minute                               int64
TimePeriod                        category
HourMinute                 timedelta64[ns]
Age                                  int32
dtype: object

### 6.Store the preprocessed dataset in a separate csv

In [ ]:
raw_dataset.to_csv('../Datasets/Clustering/Bank Customer Segmentation/preprocessed_dataset.csv', index=False)